## TensorFlow 2.0

    - api 정리 </br>
    - 이거 모드 (Eager execution)
    - 전역 메커니즘 제거 (no more globals)
    - 세션을 대신하는 함수 (Functions, not Sessions)
    
    
    * 이거 모드
    
    - 기존 Tensorflow는 연산의 결과를 알기 위해 연산 그래프를 만든 후 session.run()를 통해 실행해야 했음
    - 2.0은 파이썬과 동일한 이거 모드로 실행 되므로 연산을 구성하며 바로바로 값을 확인할 수 있음!
    
   
    * 모델 구축
    
    - 텐서플로 2.0에서는 케라스를 활용해 모델을 구축하고 학습하길 권장함
    
    1) Sequential API 
    2) Functional API
    3) Functional / Sezuential API + Custom Layers
    4) Subclassing (Custom Model)
    
        + 이 책은 주로 Subclassing을 이용할 것이다.
        

### Sequential API

    - 케라스를 활용해 가장 간단한 형태의 모델을 구축할 수 있는 API
    - 순차적인 레이어의 스택 구현 가능

In [2]:
import tensorflow as tf
from tensorflow.keras import layers

model = tf.keras.Sequential()
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(10, activation = 'softmax'))

* 레이어를 순차적으로 더하기만 하면 모델이 완성된다.

* Sequential 모듈의 경우 위와 같이 구현 자체가 매우 간단함

* 하지만 모델 층들이 순차적으로 구성되어 있지 않은 경우에는 Sequential 모듈을 사용해 구현하기 어려울 수 있다
    - 예를 들면 VQA(Visual Question Answering) 문제 
        : 사진 데이터에서 특징을 뽑는 레이어와 질문 텍스트 데이터에서 특징을 뽑는 두 레이어가 각기 순차적으로 존재 </br>
         따라서 최종 출력값을 뽑기 위해서는 이 두 값을 합쳐야 하는데 Sequential 모듈로는 두 개의 값을 합칠 수가 없기 때문에</br>
         여러 제약이 존재한다.
         
         
* Sequential 모듈을 사용하기 어려운 경우

    - 다중 입력값 모델 (Multi-input models)
    - 다중 출력값 모델 (Multi-output models)
    - 공유 층을 활용하는 모델 (Models with shared layers)
    - 데이터의 흐름이 순차적이지 않은 모델 (Models with non-sequential data flows)

### Functional API

    * 위에 나열한 Sequential 모듈을 사용하기 어려운 경우 Functional 모듈 혹은 Subclassing 방식을 사용하는 것이 적절할 수 있다.




In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

x = layers.Dense(64,activation = 'relu')(inputs)
x = layers.Dense(64, activation = 'relu')(x)
predictions = layers.Dense(10, activation ='softmax')(x)

 * Functional API를 활용하기 위해서는 입력밧을 받는 Input 모듈을 선언해야 한다.
 
 * 이 모듈을 선언할 때는 모델의 입력으로 받는 값의 형태(shape)를 정의하면 된다.
 
 * 이 Input 모듈을 정의한 후 입력값을 적용할 레이어를 호출할 때 인자로 전달하는 방식으로 구현하면 된다.
 
 * 이처럼 정의한 후 최종 출력값을 사용해 모델을 학습하면 된다. 

### Custom Layer

* 앞에서는 케라스의 layers 패키지에 정의된 레이어를 사용해 구현했다.

* 때때로 새로운 연산을 하는 레이어 혹은 편의를 위해 여러 레이어를 하나로 묶은 레이어를 구현해야 하는 경우가 있다.</br>
    : 이 때 사용자 정의 층 (custom layer)을 만들어 사용하면 된다.
    
    
* 앞서 정의한 모델에서는 dense 층이 여러 번 사용된 신경망을 사용했다.</br>
    -> 이 신경망을 하나의 레이어로 묶어 재사용을 높이고 싶다면 다음과 같이 새로운 사용자 정의 층으로 정의하면 된다.

In [ ]:
class CustomLayer(layers.Layer):
    
    def __init__(self, hidden_dimension, hidden_dimension2, output_dimension):
        self.hidden_dimension = hidden_dimension
        self.hidden_dimension2 = hidden_dimension2
        self.output_dimension = ouput_dimension
        super(CustomLayer, self).__init__()
        
        
    def build(self, input_shape):
        self.dense_layer1 = layers.Dense(self.hidden_dimension, activation = 'relu')
        self.dense_layer2 = layers.Dense(self.hidden_dimension, activation = 'relu')
        self.dense_layer3 = layers.Dense(self.output_dimension, activation = 'softmax')
        
    def call(self, inputs):
        x = self.dense_layer1(inputs)
        x = self.dense_layer2(x)
        
        
        return self.dense_layer3(x)

* 사용자 정의 층을 정의할 때 : layers 패키지의 Layer 클래스를 상속받고 위와 같이 3개의 메서드를 정의
</br></br>
* 우선 하이퍼파라미터는 객체를 생성할 때 호출되도록 __init__ 메서드에서 정의하고 
</br></br>
* 모델의 가중치와 관련된 값은 build 메서드에서 생성되도록 정의한다.
</br></br>
* 그리고 이렇게 정의한 값들을 이용해 call 메서드에서 해당 층의 로직을 정의하면 된다.
</br></br>
* 이렇게 정의한 사용자 정의 층은 Sequential API나 Functional API를 활용할 때 하나의 층으로 사용할 수 있다.
</br></br>
* 만약 Sequential 모듈을 활용한다면 아래와 같이 사용하면 됨 

In [ ]:
from tensorflow.keras import layers

model = tf.kerasSequential()
model.add(CustomLayer(64,64,10))